理论认为距离消费终端越近，价格的传导就越慢，而白条和生猪的价格以及之间的差价反应了猪肉行业“增加值”在不同产业链的分布情况，也反映了中间环节在传递供需双方信息时的倾向。从数据分类中有探讨到毛白价差的变化引起来源不同，代表了市场中供需的不同变化，而不同的供需变化又会影响到后面的价格。那么毛白价差的变化是否会影响到后期猪价变化，用格兰杰因果检验测试：

第一步：进行ADF检验和协整检验，如果都不通过则进行差分，以便后面进行VAR确定最优滞后阶数。论认为距离消费终端越近，价格的传导就越慢，而白条和生猪的价格以及之间的差价反应了猪肉行业“增加值”在不同产业链的分布情况，也反映了中间环节在传递供需双方信息时的倾向。从数据分类中有探讨到毛白价差的变化引起来源不同，代表了市场中供需的不同变化，而不同的供需变化又会影响到后面的价格。那么毛白价差的变化是否会影响到后期猪价变化，用格兰杰因果检验测试：

第一步：进行ADF检验和协整检验，如果都不通过则进行差分，以便后面进行VAR确定最优滞后阶数。

In [1]:
from statsmodels.tsa.stattools import grangercausalitytests
from statsmodels.tsa.api import VAR
from statsmodels.tsa.stattools import adfuller
import pandas as pd
import numpy as np

In [43]:
# 引入Elite日度毛白价差和金猪日度生猪均价
dataPigPrice = pd.read_excel(r"D:\git\新建文件夹\swine-data-analysis\data_clear_and_analysis\database\GL_20230227.xlsx", sheet_name="出栏价格")
dataPigPriceAndBarPrice = pd.read_excel(r"database/ELite_20230320.xlsx", sheet_name="白毛价差")

In [44]:
# 取生猪价格和毛白价差连接，并更改列名
dataPigPrice = dataPigPrice.iloc[:, 1:3]
dataPigPrice = dataPigPrice.drop(index = dataPigPrice.index[[0,1]])
dataPigPriceAndBarPrice = dataPigPriceAndBarPrice.drop(index = dataPigPriceAndBarPrice.index[[0]])

In [45]:
dataPigPrice.columns = ['Time', 'PigPrice']
dataPigPriceAndBarPrice.columns = ['Time', '毛白价差']
dataPigPrice['PigPrice'].fillna(method='bfill', inplace=True)

In [46]:
dataPigPriceAndBarPrice['毛白价差'].fillna(method='bfill', inplace=True)

In [50]:
dataUse = dataPigPriceAndBarPrice.set_index('Time').join(dataPigPrice.set_index('Time'))
dataUseNoNan = dataUse.dropna()

In [51]:
dataUseNoNan

,毛白价差,PigPrice
Time,,
2023-03-10,3.95,15.57
2023-03-09,3.96,15.57
2023-03-08,3.89,15.75
2023-03-07,3.80,15.83
2023-03-06,3.85,15.68
...,...,...
2018-01-08,5.18,15.34
2018-01-05,5.23,15.12
2018-01-04,5.23,15.06


In [53]:
print(adfuller(dataUseNoNan['PigPrice']))
print(adfuller(dataUseNoNan['毛白价差']))

(-1.6236178509391657, 0.4708331120564497, 9, 1283, {'1%': -3.435457117423131, '5%': -2.863795358068796, '10%': -2.567970771161219}, 869.9049156290052)
(-1.9750178752123888, 0.29763315318814504, 0, 1292, {'1%': -3.4354214704688113, '5%': -2.863779629316101, '10%': -2.567962394851863}, 55.056868374694204)


从生猪价格的adf检验结果来说不能拒绝原假设，所以为非平稳序列，需要进行差分后得到平稳序列

In [55]:
# 将生猪价格和毛白价差进行差分
pigPrice_transform1 = dataUseNoNan['PigPrice'].diff().dropna()
pigPriceAndBar_transform1 = dataUseNoNan['毛白价差'].diff().dropna()

In [56]:
print(adfuller(pigPrice_transform1))
print(adfuller(pigPriceAndBar_transform1))

(-8.298303351711777, 4.0829585649289577e-13, 8, 1283, {'1%': -3.435457117423131, '5%': -2.863795358068796, '10%': -2.567970771161219}, 870.913301365018)
(-36.44413153722199, 0.0, 0, 1291, {'1%': -3.4354254066484664, '5%': -2.863781366113082, '10%': -2.56796331977745}, 59.53633596673217)


经过1阶差分后两个序列在adf检验下呈平稳状态，可以进行VAR寻找最优滞后阶数